# Dataset exploration for max-min coordinates boundaries


A common normalization is to perform a dataset-wise min-max step for the skeleton joint coordinates. Our strategy is to first translate the camera coordinates to a new coordinate system. This new coordinate system corresponds to the center of the main subject's body. After performing this prior normalization, we calculate the min and max values across all joints, subjects and sequences.


In [1]:
import h5py
import numpy as np
import os

from src.utils.joints import *

# Global variables (change depending on your setup)
project_dir = os.getcwd() + '/../'
h5_dataset_path = project_dir + "/data/processed/"
skeleton_dataset_file_name = "skeleton.h5"
log_file_name = "samples_names.txt"

## 1. Open h5 skeleton dataset

In [2]:
# Take all sample names (56 880)
samples_names_list = [line.rstrip('\n') for line in open(h5_dataset_path + log_file_name)]

# Open dataset
dataset = h5py.File(h5_dataset_path + skeleton_dataset_file_name, 'r')

## 2. Calculate min-max coordinates

In [3]:
# Loop through dataset to find c_min and c_max (see VA-LSTM/CNN paper for details)

c_min = []
c_max = []

for sample_name in samples_names_list:
    # Retrieve np array
    skeleton = dataset[sample_name]["skeleton"][:]  # shape (3, max_frame, num_joint=25, 2)
    
    # Perform normalization step
    trans_vector = skeleton[:, 0, Joints.SPINEMID, :] # shape (3, 2)
    trans_vector[:, 1] = trans_vector[:, 0]
    skeleton = (skeleton.transpose(1, 2, 0, 3) - trans_vector).transpose(2, 0, 1, 3)
    
    # Update c_min and c_max
    c_min.append(np.amin(skeleton))
    c_max.append(np.amax(skeleton))

print("c_min : " + str(np.amin(c_min)))
print("c_max : " + str(np.amax(c_max)))
    

c_min : -4.766
c_max : 5.1879997


In [4]:
dataset.close()